In [ ]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import time

import random

In [ ]:
import os
import re
import sys
from google.colab import drive

In [ ]:
import copy

In [ ]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

**1. шифра Цезаря**

In [ ]:
def Сaesar(string, num):
    output = ''
    for c in string:
        if c.isalpha():
            new_num = ord(c) + num
            if new_num > ord('я'):
                new_num -= 32
            output += chr(new_num)
        else:
            output += c
    return output

In [ ]:
Сaesar('яма', 2)

'бов'

**2. Исходные данные**

In [ ]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/DOM')

In [ ]:
with open ('/content/gdrive/MyDrive/Colab Notebooks/DOM/NETOLOGIA/Золотой_теленок.txt',encoding='cp1251') as f:
  text = f.read().lower()
print (len(text))
text = re.sub('[^а-я ]', '', text)
text = re.sub('\s+', ' ', text)

617314


In [ ]:
txt = []
for i in range(int(len(text) / 50)):
    txt.append(text[i * 50:(i + 1) * 50])

In [ ]:
txt[:2]

['пешеходов надо любить пешеходы составляют большую ',
 'часть человечества мало тоголучшую его часть пешех']

**3. Подготовка данных**

In [ ]:
df = pd.DataFrame(txt)

In [ ]:
df = df.rename(columns={0: "Текст_нарезка_50"})

In [ ]:
df.head(5)

,Текст_нарезка_50
0,пешеходов надо любить пешеходы составляют боль...
1,часть человечества мало тоголучшую его часть п...
2,оды создали мир это они построили города возве...
3,ногоэтажные здания провели канализацию и водоп...
4,д замостили улицы и осветили их электрическими...


In [ ]:
df.columns

Index(['Текст_нарезка_50'], dtype='object')

In [ ]:
df['смещение']   = [random.randint(1, 10) for i in range(len(df))]
df['Метка']= df.loc[:, ['Текст_нарезка_50', 'смещение']].apply(
                        lambda row: Сaesar(row['Текст_нарезка_50'], row['смещение']), axis=1)
df.head()

,Текст_нарезка_50,смещение,Метка
0,пешеходов надо любить пешеходы составляют боль...,5,фкэкъуйуз тейу ргжнчб фкэкъуйа цуцчезрдгч журб...
1,часть человечества мало тоголучшую его часть п...,5,ьецчб ькрузкькцчзе серу чуиуршьэшг киу ьецчб ф...
2,оды создали мир это они построили города возве...,5,уйа цумйерн снх вчу утн фуцчхунрн иухуйе зумзк...
3,ногоэтажные здания провели канализацию и водоп...,5,туиувчелтак мйетнд фхузкрн петернмеынг н зуйуф...
4,д замостили улицы и осветили их электрическими...,2,ж йворуфкнк хнкшэ к рудзфкнк кч янзмфткщзумкок...


In [ ]:
print(df.iloc[:9502,:].shape)
df = df.iloc[:9502,:]

(9502, 3)


In [ ]:
train, test = train_test_split(df, test_size=0.2)
train_text = [[c for c in ph] for ph in train['Метка'] if type(ph) is str]
train_label= [[c for c in ph] for ph in train['Текст_нарезка_50'] if type(ph) is str]
test_text   = [[c for c in ph] for ph in test['Метка'] if type(ph) is str]
test_label  = [[c for c in ph] for ph in test['Текст_нарезка_50'] if type(ph) is str]

In [ ]:
len(train_text)

7601

In [ ]:
MAX_LEN       = 50
ABC           = ['none'] + [w for w in set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя ')]
CHAR_TO_INDEX = {w: i for i, w in enumerate(ABC)}
INDEX_TO_CHAR = {i: w for i, w in enumerate(ABC)}

def convert_to_torch(text):
    output = torch.zeros((len(text), MAX_LEN), dtype=int)
    for i in range(len(text)):
        for j, w in enumerate(text[i]):
            if j >= MAX_LEN:
                break
            output[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

    return output

X_train= convert_to_torch(train_text)
Y_train= convert_to_torch(train_label)
X_test = convert_to_torch(test_text)
Y_test = convert_to_torch(test_label)

**4. Задаем параметры обучения**

In [ ]:
class RNN_Network(torch.nn.Module):
    def __init__(self):
        super(RNN_Network, self).__init__()
        self.embeddings = torch.nn.Embedding(len(ABC), 28)
        self.rnn = torch.nn.RNN(28, 256, batch_first=True)
        self.linear = torch.nn.Linear(256, len(ABC))

    def forward(self, sentences, state=None):
        embds = self.embeddings(sentences)
        out, new_state = self.rnn(embds, state)
        result = self.linear(out)
        return result, new_state

In [ ]:
model = RNN_Network().to(dev)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
epochs    = 40
loss_best = 10**10

**5.  Обучение**

In [ ]:
for epoch in range(epochs):
    start = time.time()
    train_loss = 0.
    train_passed = 0
    test_loss = 0
    test_passed = 0

    for i in range(int(len(X_train) / 100)):
        X_batch = X_train[i * 100:(i + 1) * 100].to(dev)
        Y_batch = Y_train[i * 100:(i + 1) * 100].flatten().to(dev)
        model.train()
        optimizer.zero_grad()
        answers, _ = model.forward(X_batch)
        answers = answers.view(-1, len(ABC))
        loss = criterion(answers, Y_batch).to(dev)

        if loss < loss_best:
            model_best = copy.copy(model)#.to(dev)
            loss_best = loss#.to(dev)

        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_passed += 1

    with torch.no_grad():
        model.eval()
        answers, _ = model.forward(X_test.to(dev))
        answers = answers.view(-1, len(ABC))
        loss = criterion(answers, Y_test.flatten().to(dev))
        test_loss += loss.item()
        test_passed += 1

    if epoch%1 == 0:
        print(f"Epoch {epoch}. Time: {time.time() - start:.3f}, Train loss: {train_loss / train_passed:.3f}, Test loss: {test_loss / test_passed:.6f}")

Epoch 0. Time: 0.417, Train loss: 2.685, Test loss: 2.296842
Epoch 1. Time: 0.301, Train loss: 2.076, Test loss: 1.890375
Epoch 2. Time: 0.296, Train loss: 1.778, Test loss: 1.686876
Epoch 3. Time: 0.286, Train loss: 1.625, Test loss: 1.576600
Epoch 4. Time: 0.297, Train loss: 1.519, Test loss: 1.472714
Epoch 5. Time: 0.292, Train loss: 1.418, Test loss: 1.373320
Epoch 6. Time: 0.298, Train loss: 1.324, Test loss: 1.285909
Epoch 7. Time: 0.288, Train loss: 1.245, Test loss: 1.218729
Epoch 8. Time: 0.291, Train loss: 1.186, Test loss: 1.161821
Epoch 9. Time: 0.290, Train loss: 1.129, Test loss: 1.107496
Epoch 10. Time: 0.292, Train loss: 1.080, Test loss: 1.062487
Epoch 11. Time: 0.291, Train loss: 1.032, Test loss: 1.023842
Epoch 12. Time: 0.297, Train loss: 0.986, Test loss: 0.982712
Epoch 13. Time: 0.298, Train loss: 0.943, Test loss: 0.945887
Epoch 14. Time: 0.290, Train loss: 0.901, Test loss: 0.908254
Epoch 15. Time: 0.293, Train loss: 0.861, Test loss: 0.872021
Epoch 16. Time: 0.

In [ ]:
loss_best

tensor(0.2638, device='cuda:0', grad_fn=<NllLossBackward0>)

**6. Смотрим что получилось**

In [ ]:
ZZZ = convert_to_torch([[c for c in ph] for ph in df['Метка']
                                        if type(ph) is str])


df['Расшифровка']= [''.join([INDEX_TO_CHAR[i.item()] for i
                    in model_best(ZZZ.to(dev))[0][line].argmax(dim=1).detach()])
                    for line in range(df.shape[0])]

<ipython-input-91-de88a0cda638>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Расшифровка']= [''.join([INDEX_TO_CHAR[i.item()] for i


In [ ]:
df

,Текст_нарезка_50,смещение,Метка,Расшифровка
0,пешеходов надо любить пешеходы составляют боль...,5,фкэкъуйуз тейу ргжнчб фкэкъуйа цуцчезрдгч журб...,оахеткяка иыво кюбить пешеходы составляют воль...
1,часть человечества мало тоголучшую его часть п...,5,ьецчб ькрузкькцчзе серу чуиуршьэшг киу ьецчб ф...,тасны талнвечестаа кало тоголуччую его часть п...
2,оды создали мир это они построили города возве...,5,уйа цумйерн снх вчу утн фуцчхунрн иухуйе зумзк...,оич ноеаали мир это они построили города возве...
3,ногоэтажные здания провели канализацию и водоп...,5,туиувчелтак мйетнд фхузкрн петернмеынг н зуйуф...,оогоятаеные здания провели канализацию и водоп...
4,д замостили улицы и осветили их электрическими...,2,ж йворуфкнк хнкшэ к рудзфкнк кч янзмфткщзумкок...,е далостили улицы и осветили их электрическими...
...,...,...,...,...
9497,ь и хвалил юг но улыбка снова подымала его выц...,10,ж т ямкхтх ин чш эхелфк ычшмк щшоецкхк пнш меа...,е с чвалил бг но улыбна снова подымала его выц...
9498,ие на солнце усыостап побежал к вагонам северн...,4,мй сд хтпсъй чхятхцду утейкдп о ждзтсдр хйжйфс...,ва на солнце усыостап побежал к вагонам северн...
9499,городка однако городок был пуст все его жители...,10,ншъшофк шочкфш ншъшошф лех щэыь мып пнш ртьпхт...,иотоела однако городок был пуст все его жители...
9500,и к трибуне перед которой уже сидели музыканты...,10,т ф ьътлэчп щпъпо фшьшъшу эрп ытопхт цэсефкчье...,о к урибуне перед которой уже сидели музыканты...
